# Setting Up Environment

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

import time
import pandas as pd
import re
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
import datetime as dt
import numpy as np

/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Fixing Odds Scraper

In [42]:
mma_data = pd.read_csv('mma_bets.csv', index_col = 0)
mma_data['Bet_LGBM'] = 'No Bet'

In [46]:
mma_data = mma_data.append(this_weeks_bets)
mma_data.reset_index(inplace = True, drop = True)

/var/folders/xz/nv9k4jpn7l9dtk4l7yqbl8640000gn/T/ipykernel_95824/2917002565.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mma_data = mma_data.append(this_weeks_bets)


In [47]:
mma_data

,Fighter_1,Fighter_2,Fighter_1_Odds,Fighter_2_Odds,Prediction_GB_Winner,Bet_GB,Bet_LGBM,Prediction_LGBM_Winner
0,Danaa Batgerel,Brady Hiestand,-143.0,110.0,0.000000,No bet,No Bet,NaN
1,Karine Silva,Priscila Cachoeira,-180.0,150.0,0.000000,No bet,No Bet,NaN
2,Mohammed Usman,Junior Tafa,100.0,-120.0,0.000000,No bet,No Bet,NaN
3,Karol Rosa,Norma Dumont,105.0,-125.0,0.000000,No bet,No Bet,NaN
4,Rani Yahya,Montel Jackson,470.0,-650.0,0.321987,Bet 100 on Rani Yahya,No Bet,NaN
5,Rick Glenn,Christos Giagos,-161.0,125.0,0.631292,No bet,No Bet,NaN
6,Jeremiah Wells,Matthew Semelsberger,-105.0,-118.0,0.000000,No bet,No Bet,NaN
7,Iasmin Lucindo,Brogan Walker,-357.0,272.0,0.000000,No bet,No Bet,NaN
8,Bobby Green,Jared Gordon,-275.0,220.0,0.444643,Bet 100 on Jared Gordon,No Bet,NaN
9,Brad Tavares,Bruno Silva,-164.0,133.0,0.624599,No bet,No Bet,NaN
